In [25]:
from segpy.reader import create_reader
import numpy as np
import os
import pywt

In [26]:
dataset = []

directory = r'C:\Users\user\Desktop\2D\Correlated_Shot_Gathers'

for file_name in os.listdir(directory):
    
    if file_name != ".segpy":
        
        with open(directory + "\\" + file_name, 'rb') as segy_in_file:

            seg_y_dataset = create_reader(segy_in_file, endian='<')

            for i in seg_y_dataset.trace_indexes():
                dataset.append(seg_y_dataset.trace_samples(i))
            
dataset = np.array(dataset)

In [27]:
max_dataset = np.max(dataset, axis = 1)
min_dataset = np.min(dataset, axis = 1)

In [28]:
dataset = (dataset - np.repeat(np.expand_dims(min_dataset,axis = 1), 4001, axis = 1))/(np.repeat(np.expand_dims(max_dataset,axis = 1), 4001, axis = 1) - np.repeat(np.expand_dims(min_dataset,axis = 1), 4001, axis = 1))

In [29]:
from sklearn.cluster import KMeans

def quantize(signal, levels):
    
    kmeans = KMeans(n_clusters = levels).fit(signal.reshape(-1,1))
    
    return kmeans.labels_, kmeans.cluster_centers_.flatten()

In [30]:
def get_metrics(x,x_pred):
    nmse = np.sum(np.square(x - x_pred))/np.sum(np.square(x))
    nrmse = np.sqrt(np.mean(np.square(x - x_pred)))/(np.amax(x) - np.amin(x))
    snr = -10*np.log10(nmse)
    return nmse, nrmse, snr

In [31]:
def get_probability_dictionary(symbols, levels):

    return {i:np.mean(symbols == i) for i in np.arange(levels)}

In [32]:
def get_huffman_codes(symbol_probabilities):
    
    huffman_code = {str(key):'' for key in symbol_probabilities.keys()}
    symbol_probabilities = {str(key):symbol_probabilities[key] for key in symbol_probabilities.keys()}
    
    while(len(symbol_probabilities) != 1):
        
        key1 = min(symbol_probabilities, key=symbol_probabilities.get)
        value1 = symbol_probabilities[key1]
        symbol_probabilities.pop(key1)
        
        key2 = min(symbol_probabilities, key=symbol_probabilities.get)
        value2 = symbol_probabilities[key2]
        symbol_probabilities.pop(key2)
        
        for symbol in key1.split('-'):
            huffman_code[symbol] += '1'
            
        for symbol in key2.split('-'):
            huffman_code[symbol] += '0'
            
        symbol_probabilities[key1 + '-' + key2] = value1 + value2
            
    return {int(key):huffman_code[key][::-1] for key in huffman_code.keys()}

In [64]:
def get_huffman_encoeded_length(d1_sym, M1, d2_sym, M2, d3_sym, M3, c3_sym):
    
    length = 0
    
    d1_sym_huffman_dict = get_huffman_codes(get_probability_dictionary(d1_sym, M1))
    
    for key, value in d1_sym_huffman_dict.items():
        length += np.sum(d1_sym == key)*len(value) 
        
    d2_sym_huffman_dict = get_huffman_codes(get_probability_dictionary(d2_sym, M2))
    
    for key, value in d2_sym_huffman_dict.items():
        length += np.sum(d2_sym == key)*len(value) 
        
    d3_sym_huffman_dict = get_huffman_codes(get_probability_dictionary(d3_sym, M3))
    
    for key, value in d3_sym_huffman_dict.items():
        length += np.sum(d3_sym == key)*len(value)
        
    c3_sym_huffman_dict = get_huffman_codes(get_probability_dictionary(c3_sym, M3))
    
    for key, value in c3_sym_huffman_dict.items():
        length += np.sum(c3_sym == key)*len(value)
        
    return length

In [34]:
def get_cr_and_metrics(dataset, wavelet = 'db2', M1 = 4, M2 = 8, M3 = 16):
    
    nmse, nrmse, snr = np.zeros(len(dataset)), np.zeros(len(dataset)), np.zeros(len(dataset))
    
    total_length = 0
    
    for j, sample in enumerate(dataset):
        
        c3, d3, d2, d1 = pywt.wavedec(sample, wavelet, level = 3)
        
        d1_sym, d1_val = quantize(d1, M1)
        d2_sym, d2_val = quantize(d2, M2)
        d3_sym, d3_val = quantize(d3, M3)
        c3_sym, c3_val = quantize(c3, M3)
        
        d1_reconstruct = np.array([d1_val[i] for i in d1_sym])
        d2_reconstruct = np.array([d2_val[i] for i in d2_sym])
        d3_reconstruct = np.array([d3_val[i] for i in d3_sym])
        c3_reconstruct = np.array([c3_val[i] for i in c3_sym])
        
        reconstructed_signal = pywt.waverec((c3_reconstruct,d3_reconstruct,d2_reconstruct,d1_reconstruct), wavelet)
        
        nmse[j], nrmse[j], snr[j] = get_metrics(sample, reconstructed_signal[:-1])
        
        total_length += get_huffman_encoeded_length(d1_sym, M1, d2_sym, M2, d3_sym, M3, c3_sym)
        
    return 4001*64*len(dataset)/total_length, np.mean(nmse), np.mean(nrmse), np.mean(snr)

In [60]:
get_cr_and_metrics(dataset[900:1000], 'db2', 8, 16, 64)

(22.639674423827252,
 8.632897820461949e-06,
 0.0013707870806683787,
 52.21920866966248)

Making a model that ignores d1

In [65]:
def get_huffman_encoeded_length_without_d1(d2_sym, M2, d3_sym, M3, c3_sym):
    
    length = 0
        
    d2_sym_huffman_dict = get_huffman_codes(get_probability_dictionary(d2_sym, M2))
    
    for key, value in d2_sym_huffman_dict.items():
        length += np.sum(d2_sym == key)*len(value) 
        
    d3_sym_huffman_dict = get_huffman_codes(get_probability_dictionary(d3_sym, M3))
    
    for key, value in d3_sym_huffman_dict.items():
        length += np.sum(d3_sym == key)*len(value)
        
    c3_sym_huffman_dict = get_huffman_codes(get_probability_dictionary(c3_sym, M3))
    
    for key, value in c3_sym_huffman_dict.items():
        length += np.sum(c3_sym == key)*len(value)
        
    return length

In [72]:
def get_cr_and_metrics_without_d1(dataset, wavelet = 'db2', M2 = 8, M3 = 16):
    
    nmse, nrmse, snr = np.zeros(len(dataset)), np.zeros(len(dataset)), np.zeros(len(dataset))
    
    total_length = 0
    
    for j, sample in enumerate(dataset):
        
        c3, d3, d2, d1 = pywt.wavedec(sample, wavelet, level = 3)
        d2_sym, d2_val = quantize(d2, M2)
        d3_sym, d3_val = quantize(d3, M3)
        c3_sym, c3_val = quantize(c3, M3)
        
        d2_reconstruct = np.array([d2_val[i] for i in d2_sym])
        d3_reconstruct = np.array([d3_val[i] for i in d3_sym])
        c3_reconstruct = np.array([c3_val[i] for i in c3_sym])
        
        reconstructed_signal = pywt.waverec((c3_reconstruct,d3_reconstruct,d2_reconstruct,np.zeros(2002)), wavelet)
        
        nmse[j], nrmse[j], snr[j] = get_metrics(sample, reconstructed_signal[:-1])
        
        total_length += get_huffman_encoeded_length_without_d1(d2_sym, M2, d3_sym, M3, c3_sym)
        
    return 4001*64*len(dataset)/total_length, np.mean(nmse), np.mean(nrmse), np.mean(snr)

In [73]:
get_cr_and_metrics_without_d1(dataset[900:1000], 'db2', 2, 2)

(127.64905284147558,
 0.014091713596524556,
 0.05693899028792911,
 19.56495417418365)

In [76]:
get_cr_and_metrics_without_d1(dataset[900:1000], 'db2', 2, 4)

(107.62383103919302,
 0.004737264925443791,
 0.03347772928172762,
 24.080715597525792)

In [77]:
get_cr_and_metrics_without_d1(dataset[900:1000], 'db2', 4,8)

(79.27948010613365,
 0.0011798831569598011,
 0.016570604862909784,
 30.297794314585907)

In [78]:
get_cr_and_metrics_without_d1(dataset[900:1000], 'db2', 4, 16)

(61.128882862408446,
 0.0002684953694337118,
 0.00784826569433506,
 36.82138096645946)

In [79]:
get_cr_and_metrics_without_d1(dataset[900:1000], 'db2', 8, 32)

(44.22025161251327,
 5.62403323986683e-05,
 0.0035866707405213982,
 43.60017020627727)

In [75]:
get_cr_and_metrics_without_d1(dataset[900:1000], 'db2', 16, 64)

(32.63462560489601,
 1.2989737528696998e-05,
 0.0017683824279625423,
 49.50535980326118)